In [1]:
# Transformers installation
! pip install transformers datasets
! pip install transformers datasets evaluate rouge_score
!pip install sentencepiece

# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=44da213103a25939a05841f61bba099a3ca70e5ae87041d269aa504c0a917274
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
!pip install wandb

In [3]:
!pip install huggingface-hub


In [4]:
from datasets import Dataset ,DatasetDict

In [5]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("token")
secret_value_1 = user_secrets.get_secret("wandb-api-key")

# wandb.login(key=secret_value_0)

wandb.login(key=secret_value_1)


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import huggingface_hub
huggingface_hub.login(token=secret_value_0, write_permission=True)


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from glob import glob

In [8]:
import pandas as pd
import random

# Load the dataset
filename = "/kaggle/input/merged-csv/merged_data.csv"
df = pd.read_csv(filename)

# Shuffle the dataset
df = df.sample(frac=1, random_state=42)  # Shuffle the DataFrame with a fixed random seed for reproducibility
df = df.head(10000)
# Rename the columns to 'text' and 'summary'
print(len(df))
df.rename(columns={'Input': 'text', 'Output': 'summary'}, inplace=True)

10000


In [9]:
dataset = Dataset.from_pandas(df[['text', 'summary']])

In [10]:
dataset

Dataset({
    features: ['text', 'summary', '__index_level_0__'],
    num_rows: 10000
})

In [11]:
dataset = dataset.train_test_split(test_size=0.001)

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', '__index_level_0__'],
        num_rows: 9990
    })
    test: Dataset({
        features: ['text', 'summary', '__index_level_0__'],
        num_rows: 10
    })
})

In [13]:
dataset["train"][0]['text']

'Strategy & Planning Talent & Transformation Finance, Tax & Treasury Technology Capital Allocation Communications Legal Intellectual Property www.informa.com 8 Informa PLC Annual Report 2014 ACADEMIC   PUBLISHING  p.23 BUSINESS   INTELLIGENCE  p.29 GLOBAL   EXHIBITIONS  p.35 KNOWLEDGE &   NETWORKING  p.41 WHAT IT DOES The Academic Publishing Division publishes specialist books and journals. Operating as the Taylor & Francis Group, it is recognised internationally as  one of the worlds leading education publishers through its five leading  imprints: Taylor & Francis, Routledge, CRC Press, Garland Science and  Cogent OA. Ithas a portfolio of more than 100,000 book titles and 2,100  journals available in both print and digital formats, across subject areas  within Humanities and Social Sciences, and Science, Technology  andMedicine. WHAT IT DOES The Business Intelligence Division provides specialist data,  intelligence and insight to businesses, helping them make better  decisions, gain c

The next step is to load a  tokenizer to process `text` and `summary`:

In [14]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# Load the Pegasus-XSum tokenizer
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum' , add_prefix_space = True)

# Load the Pegasus-XSum model
pegasus_model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import DataCollator

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    labels = tokenizer(examples["summary"], max_length=64, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Now you can create a custom data collator using the Python function
# 


In [16]:
# tokenized_dataset = dataset.map(preprocess_function , batched = True)
tokenized_dataset = dataset.map(preprocess_function , batched = True)


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
# import pickle

# # Assuming tokenized_dataset is your tokenized dataset
# with open('tpega.pkl', 'wb') as pkl_file:
#     pickle.dump(tokenized_dataset, pkl_file)

    


In [18]:
# import pickle

# # Assuming you have a pickle file named 'tpega.pkl'
# with open('/kaggle/input/pkl-file-pegasus/tpega.pkl', 'rb') as pkl_file:
#     loaded_tokenized_dataset = pickle.load(pkl_file)


In [19]:
# from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# # Load the Pegasus-XSum tokenizer
# tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

# # Load the Pegasus-XSum model
# pegasus_model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')


Now create a batch of examples using [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [20]:
checkpoint = "google/pegasus-xsum"

In [21]:
from transformers import DataCollatorForSeq2Seq

# Create the Pegasus-XSum tokenizer and model
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
# model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

# Create a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=pegasus_model)


## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [ROUGE](https://huggingface.co/spaces/evaluate-metric/rouge) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [22]:
import evaluate

rouge = evaluate.load("rouge")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the ROUGE metric:

In [23]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load T5 with [AutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSeq2SeqLM):

At this point, only three steps remain:

1. Define your training hyperparameters in [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the ROUGE metric and save the training checkpoint.
2. Pass the training arguments to [Seq2SeqTrainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [24]:
import torch
torch.cuda.empty_cache()



In [25]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="Google-Pegasus-XSum",
    evaluation_strategy="epoch",
    learning_rate=1e-5,  # Adjust the learning rate if needed
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    num_train_epochs=9,
    predict_with_generate=True,
    fp16=True,
    save_total_limit=2,
    save_strategy="no",  # You missed the quotation marks
    load_best_model_at_end=False,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=pegasus_model,
    args=training_args,
    data_collator=data_collator,  # Use the Pegasus-XSum data collator
    compute_metrics=compute_metrics,  # Define your custom metric function
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

trainer.train()


wandb: Currently logged in as: mister (msi). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231030_184116-g1gdxsgf
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run headless-monster-12
wandb: ⭐️ View project at https://wandb.ai/msi/huggingface
wandb: 🚀 View run at https://wandb.ai/msi/huggingface/runs/g1gdxsgf
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.629700,0.858522,0.814900,0.789100,0.806300,0.807900,62.100000
2,0.461900,0.798168,0.817900,0.792800,0.807300,0.809000,62.600000
3,0.477400,0.742990,0.736000,0.707000,0.720300,0.722000,63.000000
4,0.385000,0.714956,0.736100,0.707000,0.720600,0.722000,62.900000
5,0.405300,0.706042,0.739500,0.702600,0.722200,0.724700,62.900000
6,0.369700,0.698083,0.739500,0.702600,0.722200,0.724700,62.900000
7,0.371300,0.684598,0.739700,0.702700,0.722200,0.724900,62.900000
8,0.356400,0.681671,0.739700,0.702700,0.720600,0.723200,62.800000
9,0.348400,0.681108,0.739500,0.702600,0.722200,0.724700,62.900000


TrainOutput(global_step=22482, training_loss=0.44295669079208677, metrics={'train_runtime': 28555.6797, 'train_samples_per_second': 3.149, 'train_steps_per_second': 0.787, 'total_flos': 1.2989587274072064e+17, 'train_loss': 0.44295669079208677, 'epoch': 9.0})

Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [26]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import os

# Define the directory where you want to save the model
output_directory = "model"

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Save the model, tokenizer, and configuration to the specified directory
pegasus_model.save_pretrained(output_directory)
tokenizer.save_pretrained(output_directory)



('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/spiece.model',
 'model/added_tokens.json')

<Tip>

For a more in-depth example of how to finetune a model for summarization, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization.ipynb)
or [TensorFlow notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization-tf.ipynb).

</Tip>

In [27]:
text = dataset["test"]["text"]

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for summarization with your model, and pass your text to it:

In [28]:
# # Load the saved model and tokenizer for testing
# model = AutoModelForSeq2SeqLM.from_pretrained(output_directory)
# tokenizer = AutoTokenizer.from_pretrained(output_directory)

# # Instantiate a pipeline for summarization with the saved model
# summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# # Generate the summary
# generated_summary = summarizer(text)

# # Print the generated summary
# print(generated_summary)


In [29]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


# Define the name for your model on Hugging Face Hub
hub_model_name = "randomshit11/pegasus-model-run-main-2"

# Save the model and tokenizer to the Hugging Face Model Hub
pegasus_model.push_to_hub(hub_model_name)
tokenizer.push_to_hub(hub_model_name)

# Once the above is done, you can also save the configuration for the model
pegasus_model.config.push_to_hub(hub_model_name)

# Commit your changes
pegasus_model.push_to_hub(hub_model_name, commit_message="Initial commit")

print(f"Model and tokenizer are now available on the Hugging Face Model Hub with the name: {hub_model_name}")

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Model and tokenizer are now available on the Hugging Face Model Hub with the name: randomshit11/pegasus-model-run-main-2
